<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
diff --git a/mindspore/lite/nnacl/fp32_grad/layernorm_grad.c b/mindspore/lite/nnacl/fp32_grad/layernorm_grad.c
index 38357d3..7a834e6 100644
--- a/mindspore/lite/nnacl/fp32_grad/layernorm_grad.c
+++ b/mindspore/lite/nnacl/fp32_grad/layernorm_grad.c
@@ -19,14 +19,15 @@
 
 void LayerNormGrad(const float *x, const float *dy, const float *var, const float *mean, const float *gamma,
                    int param_num, int param_size, int block_num, int block_size, float *dx, float *dg, float *db) {
-  // var is actually 1/sqrf(var)-> var^0.5
+  // var is actually layer_norm forward output var
+  float eps = 1e-12;
   const float *var_sqrt_rev = var;
   for (size_t i = 0; i < param_num; ++i) {
     float dgamma = 0.0f;
     float dbeta = 0.0f;
     for (size_t j = i; j < param_size * param_num; j += param_num) {
       int norm_shift = (int)(j / block_size);
-      dgamma += dy[j] * var_sqrt_rev[norm_shift] * (x[j] - mean[norm_shift]);
+      dgamma += dy[j] * pow(var[norm_shift]+eps, -0.5) * (x[j] - mean[norm_shift]);
       dbeta += dy[j];
     }
     dg[i] = dgamma;
@@ -41,13 +42,15 @@ void LayerNormGrad(const float *x, const float *dy, const float *var, const floa
       int norm_shift = (int)(j / block_size);
       float dxm = x[j] - mean[norm_shift];
       float dyg = dy[j] * gamma[param_shift];
-      sum1 += -0.5f * dyg * dxm * var_sqrt_rev[norm_shift] * var_sqrt_rev[norm_shift] * var_sqrt_rev[norm_shift];
+
+      sum1 += -0.5f * dyg * dxm * pow(var_sqrt_rev[norm_shift] + eps, -1.5);
+      sum2 += dyg;
       sum3 += -2.0f * dxm;
     }
     for (size_t j = i * block_size; j < (i + 1) * block_size; ++j) {
       int param_shift = j % param_num;
       int norm_shift = (int)(j / block_size);
-      float var_sqrt = var_sqrt_rev[norm_shift];
+      float var_sqrt = pow(var_sqrt_rev[norm_shift]+eps, -0.5);
       float dx1 = dy[j] * gamma[param_shift] * var_sqrt;
       float dx2 = sum1 * 2.0f / block_size * (x[j] - mean[norm_shift]);
       float dx3 = (-1.0f * var_sqrt * sum2 + (1.0f / block_size) * sum1 * sum3) * (1.0f / block_size);
